Notes from https://algotrading101.com/learn/binance-python-api-guide/

In [2]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import os
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

client = Client(api_key, api_secret)
#Set the following for test only:
#client.API_URL = 'https://testnet.binance.vision/api'
#client.get_account()

In [6]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
# print full output (dictionary)
print(btc_price)

{'symbol': 'BTCUSDT', 'price': '17076.15000000'}


In [3]:
from time import sleep

from binance import ThreadedWebsocketManager
btc_price = {'error':False}

In [7]:
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['last'] = msg['a']
        btc_price['error'] = False
    else:
        btc_price['error'] = True

In [8]:
# init and start the WebSocket
bsm = ThreadedWebsocketManager()
bsm.start()
# subscribe to a stream
bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')


'btcusdt@ticker'

17060.76000000
17061.16000000
17063.35000000
17063.59000000
17064.04000000
17065.45000000
17067.25000000
17066.67000000


In [9]:
bsm.stop()

17065.34000000


In [7]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
print(timestamp)

1502928000000


In [1]:
# request historical candle (or klines) data
bars = client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=1000)
# delete unwanted data - just keep date, open, high, low, close
for line in bars:
    del line[5:]

NameError: name 'client' is not defined

In [9]:
import pandas as pd
btc_df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close'])
btc_df.set_index('date', inplace=True)
print(btc_df.head())
btc_df.to_csv('btc_bars3.csv')

                        open           high            low          close
date                                                                     
1502928000000  4261.48000000  4485.39000000  4200.74000000  4285.08000000
1503014400000  4285.08000000  4371.52000000  3938.77000000  4108.37000000
1503100800000  4108.37000000  4184.69000000  3850.00000000  4139.98000000
1503187200000  4120.98000000  4211.08000000  4032.62000000  4086.29000000
1503273600000  4069.13000000  4119.62000000  3911.79000000  4016.00000000


/Users/philipmattocks/binance-python-work/binance-venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


see https://data.binance.vision/ for more market data